In [42]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from ipywidgets import interact

# Size of the 2D grid
N = 200

# Time step
dt = 1.0

# Initialize U and V
U = np.ones((N, N))  # Initial condition for U
V = np.zeros((N, N))  # Initial condition for V

# Initialize small area in center of V = 1
v_size = 2
V[N//2-v_size:N//2+v_size, N//2-v_size:N//2+v_size] = 1

# Parameters
Du, Dv, F, k = 1.0, 0.5, 0.0545, 0.062  # Adjusted parameters for more dynamic pattern

# Function to calculate Laplacian using convolution
def laplacian(Z):
    laplacian_kernel = np.array([[.05, .2, .05],
                                 [.2, -1, .2],
                                 [.05, .2, .05]])
    return signal.convolve2d(Z, laplacian_kernel, mode='same', boundary='wrap')

# Store each frame of the simulation
V_frames = []
U_frames = []

# Run the simulation
for i in range(10000):  # 10000 steps
    U_new = U + dt * (Du * laplacian(U) - U*V*V + F*(1-U))
    V_new = V + dt * (Dv * laplacian(V) + U*V*V - (F+k)*V)

    # Clip values to keep within the range of 0 and 1
    U_new = np.clip(U_new, 0, 1)
    V_new = np.clip(V_new, 0, 1)

    U, V = U_new, V_new
    if i % 100 == 0:  # Store every 100th frame
        V_frames.append(V.copy())
        U_frames.append(U.copy())

# Function to display a frame
def display_frame(i):
    fig, axs = plt.subplots(1, 2, figsize=(6, 3))

    axs[0].imshow(V_frames[i], cmap='hot', interpolation='nearest')
    axs[0].set_title('V')

    axs[1].imshow(U_frames[i], cmap='hot', interpolation='nearest')
    axs[1].set_title('U')

    plt.tight_layout()
    plt.show()

# Use a slider to navigate through the frames
interactive_plot = interact(display_frame, i=(0, len(V_frames)-1))


interactive(children=(IntSlider(value=49, description='i', max=99), Output()), _dom_classes=('widget-interact'…